Seperated healthy cough audios from the data folder coswera as its needed for healthy class

In [8]:
import pandas as pd
import os
import shutil
from pathlib import Path

# === CONFIGURATION ===
csv_file = "combined_data.csv"  # Save your Excel as this CSV in the same folder
extracted_data_root = Path("F:/Research_Project_CMP9140/Data_Set/Coswara-Data/Coswara-Data-master/Extracted_data")
output_healthy = Path("data/raw/healthy")

output_healthy.mkdir(parents=True, exist_ok=True)

# === LOAD METADATA ===
df = pd.read_csv(csv_file)

# Filter only healthy participants
df_healthy = df[df["covid_status"].str.lower() == "healthy"]

print(f"[INFO] Total healthy participants: {len(df_healthy)}")

for idx, row in df_healthy.iterrows():
    uid = row["id"]

    user_folder = None
    # Search inside each date-named subfolder
    for date_folder in extracted_data_root.iterdir():
        possible_path = date_folder / uid
        if possible_path.exists():
            user_folder = possible_path
            break

    if user_folder is None:
        print(f"[Skip] Folder not found for user ID: {uid}")
        continue

    # Copy cough files only (heavy + shallow)
    for cough_file in ["cough-heavy.wav", "cough-shallow.wav"]:
        source = user_folder / cough_file
        if source.exists():
            dest = output_healthy / f"{uid}_{cough_file}"
            shutil.copy(source, dest)
            print(f"[Copied] {dest}")
        else:
            print(f"[Missing] {cough_file} for {uid}")


[INFO] Total healthy participants: 1433
[Copied] data\raw\healthy\iV3Db6t1T8b7c5HQY2TwxIhjbzD3_cough-heavy.wav
[Copied] data\raw\healthy\iV3Db6t1T8b7c5HQY2TwxIhjbzD3_cough-shallow.wav
[Copied] data\raw\healthy\AxuYWBN0jFVLINCBqIW5aZmGCdu1_cough-heavy.wav
[Copied] data\raw\healthy\AxuYWBN0jFVLINCBqIW5aZmGCdu1_cough-shallow.wav
[Copied] data\raw\healthy\C5eIsssb9GSkaAgIfsHMHeR6fSh1_cough-heavy.wav
[Copied] data\raw\healthy\C5eIsssb9GSkaAgIfsHMHeR6fSh1_cough-shallow.wav
[Copied] data\raw\healthy\YjbEAECMBIaZKyfqOvWy5DDImUb2_cough-heavy.wav
[Copied] data\raw\healthy\YjbEAECMBIaZKyfqOvWy5DDImUb2_cough-shallow.wav
[Copied] data\raw\healthy\aGOvk4ji0cVqIzCs1jHnzlw2UEy2_cough-heavy.wav
[Copied] data\raw\healthy\aGOvk4ji0cVqIzCs1jHnzlw2UEy2_cough-shallow.wav
[Copied] data\raw\healthy\rB5oGtrGYZR5uJUXEaDYrrredz13_cough-heavy.wav
[Copied] data\raw\healthy\rB5oGtrGYZR5uJUXEaDYrrredz13_cough-shallow.wav
[Copied] data\raw\healthy\OW5RTM4WXPawz0QLpsfjsl4FqM22_cough-heavy.wav
[Copied] data\raw\healthy

In [ ]:
import pandas as pd
from pathlib import Path
import shutil

# === CONFIG (Update these paths) ===
csv_file = "F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012/metadata_compiled.csv"
audio_dir = Path("F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012")
output_healthy_audio = Path("../data/raw/healthy2/")
# ==================================

# Load metadata and set UUID as the index for very fast lookups
print("[INFO] Loading metadata file...")
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()
# This helps find files even if there are extra spaces
df['uuid'] = df['uuid'].str.strip() 
df.set_index('uuid', inplace=True)
print("[INFO] Metadata loaded.")

# Get a list of all .wav files you have actually downloaded
print("[INFO] Finding all downloaded .wav files...")
all_wav_files = list(audio_dir.glob("**/*.wav"))
print(f"[INFO] Found {len(all_wav_files)} .wav files to check.")

copied_count = 0
# Loop through the list of audio files that actually exist on your computer
for audio_path in tqdm(all_wav_files, desc="Separating healthy files"):
    # Get the filename without the ".wav" extension
    file_id = audio_path.stem 

    try:
        # Look up the status of this specific file_id in the CSV
        status = df.loc[file_id, 'status']
        
        # If the status is 'healthy', copy the file
        if status.lower() == 'healthy':
            dest_path = output_healthy_audio / audio_path.name
            shutil.copy(audio_path, dest_path)
            copied_count += 1
    except KeyError:
        # This means the audio file's ID wasn't in the CSV, so we skip it
        continue
    except Exception as e:
        print(f"An error occurred with {file_id}: {e}")

print(f"\n[COMPLETE] Copied {copied_count} healthy audio files to the 'raw/healthy' folder.")


In [13]:
import pandas as pd
from pathlib import Path
import shutil
from tqdm import tqdm

# === CONFIG (Final Check) ===
# 1. Make sure this points to the main metadata CSV file
csv_file = "F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012/metadata_compiled.csv"
# 2. Make sure this points to the PARENT COUGHVID folder
audio_dir = Path("F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012/metadata_compiled.csv")
# 3. This path is for the output
output_healthy_audio = Path("../data/raw/healthy3/")
# ==================================

output_healthy_audio.mkdir(parents=True, exist_ok=True)

# Load metadata and set UUID as the index for fast lookup
print("[INFO] Loading metadata file...")
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()
df['uuid'] = df['uuid'].str.strip() 
df.set_index('uuid', inplace=True)
print("[INFO] Metadata loaded.")

# Get a list of all .wav files in ALL subdirectories
print("[INFO] Finding all downloaded .wav files...")
all_wav_files = list(audio_dir.glob("**/*.wav"))
print(f"[INFO] Found {len(all_wav_files)} .wav files to check.")

copied_count = 0
# Loop through the files that actually exist on your computer
for audio_path in tqdm(all_wav_files, desc="Separating healthy files"):
    file_id = audio_path.stem 

    try:
        # Look up the status of this specific file_id in the CSV
        status_value = df.loc[file_id, 'status']
        
        # This is the robust check. It handles all data types.
        if isinstance(status_value, str) and status_value.lower() == 'healthy':
            dest_path = output_healthy_audio / audio_path.name
            shutil.copy(audio_path, dest_path)
            copied_count += 1
            
    except KeyError:
        # This uuid was not found in the CSV file, so we skip it
        continue
    except Exception as e:
        # This will catch any other unexpected errors
        print(f"An error occurred with {file_id}: {e}")

print(f"\n[COMPLETE] Copied {copied_count} healthy audio files to the 'raw/healthy' folder.")

<>:8: SyntaxWarning: invalid escape sequence '\R'
<>:10: SyntaxWarning: invalid escape sequence '\R'
<>:8: SyntaxWarning: invalid escape sequence '\R'
<>:10: SyntaxWarning: invalid escape sequence '\R'
C:\Users\HP\AppData\Local\Temp\ipykernel_23352\2566748600.py:8: SyntaxWarning: invalid escape sequence '\R'
  csv_file = "F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012/metadata_compiled.csv"
C:\Users\HP\AppData\Local\Temp\ipykernel_23352\2566748600.py:10: SyntaxWarning: invalid escape sequence '\R'
  audio_dir = Path("F:\Research_Project_CMP9140\Data_Set\COUGHVID\coughvid_20211012/metadata_compiled.csv")


[INFO] Loading metadata file...
[INFO] Metadata loaded.
[INFO] Finding all downloaded .wav files...
[INFO] Found 0 .wav files to check.


Separating healthy files: 0it [00:00, ?it/s]


[COMPLETE] Copied 0 healthy audio files to the 'raw/healthy' folder.
